Design of a new game, Spuds

In [1]:
using Random
using DataFrames
using CSV
using Statistics
using Printf
using LinearAlgebra

In [2]:
struct Spud
    name::String
    h::Int64
    f::Int64
    l::Int64
    p::Int64
    r::Int64
    s::Int64
end

const MXS = 100
const MXV = 20
const MNV = 1

function cost(h::Int64, f::Int64, l::Int64, p::Int64, r::Int64, s::Int64)::Int64
    #return h*(f+l-2*MNV) + f*(p+r+s-3*MNV) + h+f+l+p+r+s-6*MNV
    return h*(f+l) + f*(p+r+s)
end

cost (generic function with 1 method)

In [3]:
function eval_battle(a::Spud, b::Spud)::Int
    a_finds = a.f >= b.h
    b_finds = b.f >= a.h
    #score_a = 4 * (1000*a.p + a.l > 1000*b.p+b.l) + 3 * (1000*a.r+a.l > 1000*b.r+b.l) + 2 * (1000*a.s+a.l > 1000*b.s+b.l)
    #score_b = 4 * (1000*a.p + a.l < 1000*b.p+b.l) + 3 * (1000*a.r+a.l < 1000*b.r+b.l) + 2 * (1000*a.s+a.l < 1000*b.s+b.l)
    score_a = 4 * (a.p > b.p) + 3 * (a.r > b.r) + 2 * (a.s > b.s)
    score_b = 4 * (a.p < b.p) + 3 * (a.r < b.r) + 2 * (a.s < b.s)
    melee_win = sign(score_a - score_b)
    if a_finds && b_finds
        tiebreaker = 4 * melee_win + 2 * sign(a.l - b.l) + sign(a.h - b.h)
        return sign(tiebreaker)
    end
    if a_finds && !b_finds
        return 1
    end
    if !a_finds && b_finds
        return -1
    end
    if !a_finds && !b_finds
        tiebreaker = 4 * sign(a.l - b.l) + 2 * melee_win + sign(a.f - b.f)
        return sign(tiebreaker)
    end
end


eval_battle (generic function with 1 method)

In [4]:
tab = CSV.read("census_yob2022_names.txt", DataFrame, header = false)
names = tab.Column1
adjectives = CSV.read("adjectives.csv", DataFrame)
nouns = CSV.read("nouns.csv", DataFrame)
jobs = CSV.read("jobs.csv", DataFrame)

const mult_noun = 1
const mult_adj = 2
const mult_job = 3


function random_name_and_stat()::Spud
    vp = [0, 0, 0, 0, 0, 0]
    #nametype = rand([1,1,1,1,1,2,2,2,3])
    name = ""
    noun = ""
    adj = ""
    noun_i = rand(1:nrow(nouns))
    noun = nouns[noun_i, :noun]
    vp[1] = vp[1] + nouns[noun_i, :H] * mult_noun
    vp[2] = vp[2] + nouns[noun_i, :F] * mult_noun
    vp[3] = vp[3] + nouns[noun_i, :L] * mult_noun
    vp[4] = vp[4] + nouns[noun_i, :P] * mult_noun
    vp[5] = vp[5] + nouns[noun_i, :R] * mult_noun
    vp[6] = vp[6] + nouns[noun_i, :S] * mult_noun
    adj_i = rand(1:nrow(adjectives))
    adj = adjectives[adj_i, :adjective]
    vp[1] = vp[1] + adjectives[adj_i, :H] * mult_adj
    vp[2] = vp[2] + adjectives[adj_i, :F] * mult_adj
    vp[3] = vp[3] + adjectives[adj_i, :L] * mult_adj
    vp[4] = vp[4] + adjectives[adj_i, :P] * mult_adj
    vp[5] = vp[5] + adjectives[adj_i, :R] * mult_adj
    vp[6] = vp[6] + adjectives[adj_i, :S] * mult_adj
    job_i = rand(1:nrow(jobs))
    job = jobs[job_i, :job]
    vp[1] = vp[1] + jobs[job_i, :H] * mult_job
    vp[2] = vp[2] + jobs[job_i, :F] * mult_job
    vp[3] = vp[3] + jobs[job_i, :L] * mult_job
    vp[4] = vp[4] + jobs[job_i, :P] * mult_job
    vp[5] = vp[5] + jobs[job_i, :R] * mult_job
    vp[6] = vp[6] + jobs[job_i, :S] * mult_job
    name = string(adj, " ", job, " ", noun)
    Spud(name, vp[1], vp[2], vp[3], vp[4], vp[5], vp[6])
end

random_name_and_stat (generic function with 1 method)

In [5]:
function rand_rename(a::Spud, n_tries::Int = 100)::Spud
    best_score = 0.0
    best_b = random_name_and_stat()
    for ii in 1:n_tries
        b = random_name_and_stat()
        b_norm = sqrt(b.h^2 + b.f^2 + b.l^2 + b.p^2 + b.r^2 + b.s^2)
        score = (a.h * b.h + a.f * b.f + a.l * b.l + a.p * b.p + a.r * b.r + a.s * b.s)/b_norm
        if score > best_score
            best_score = score
            best_b = b
        end
    end
    return Spud(best_b.name, a.h, a.f, a.l, a.p, a.r, a.s)
end

rand_rename (generic function with 2 methods)

In [6]:
function eval_battle_list(a::Spud, bs::Array{Spud})::Int
    score = 0
    for ii in 1:length(bs)
        score = score + eval_battle(a, bs[ii])
    end
    return score
end

function eval_battle_list2(a::Spud, bs::Array{Spud}, w::Vector{Float64})::AbstractFloat
    score = 0.0
    for ii in 1:length(bs)
        score = score + w[ii] * eval_battle(a, bs[ii])
    end
    return score
end


function pick_best(as::Array{Spud}, bs::Array{Spud})::Spud
    bestscore = -999
    bestf = as[1]
    for ii in 1:length(as)
        score = eval_battle_list(as[ii], bs)
        if score > bestscore
            bestscore = score
            bestf = as[ii]
        end
    end
    return bestf
end

function pick_best_rdmly(as::Array{Spud}, bs::Array{Spud}, ntries::Int)::Spud
    bestscore = -999
    bestf = rand(as)
    for ii in 1:ntries
        f = rand(as)
        score = eval_battle_list(f, bs)
        if score > bestscore
            bestscore = score
            bestf = f
        end
    end
    return bestf
end

function pick_best_rdmly_g(g::Function, bs::Array{Spud}, ntries::Int)::Spud
    bestscore = -999
    bestf = g()
    for ii in 1:ntries
        f = g()
        score = eval_battle_list(f, bs)
        if score > bestscore
            bestscore = score
            bestf = f
        end
    end
    return bestf
end

function spuds_to_df(as::Array{Spud})::DataFrame
    names = Array{String}(undef, length(as))
    hs = Array{Int}(undef, length(as))
    fs = Array{Int}(undef, length(as))
    ls = Array{Int}(undef, length(as))
    ps = Array{Int}(undef, length(as))
    rs = Array{Int}(undef, length(as))
    ss = Array{Int}(undef, length(as))
    for ii in 1:length(as)
        names[ii] = as[ii].name
        hs[ii] = as[ii].h
        fs[ii] = as[ii].f
        ls[ii] = as[ii].l
        ps[ii] = as[ii].p
        rs[ii] = as[ii].r
        ss[ii] = as[ii].s
    end
    df = DataFrame(name = names, h = hs, f = fs, l = ls, p = ps, r = rs, s = ss)
    return df
end

function fpart(x::AbstractFloat)::AbstractFloat
  return x - trunc(x)
end

function eval_team_battle(as::Array{Spud}, bs::Array{Spud})::Int
    a_i = 1
    b_i = 1
    while (a_i <= length(as)) && (b_i <= length(bs))
        res = eval_battle(as[a_i], bs[b_i])
        if res == 1
            b_i = b_i + 1
        else
            a_i = a_i + 1
            if res == 0
                b_i = b_i + 1
            end
        end
    end
    a_out = (a_i > length(as))
    b_out = (b_i > length(as))
    if a_out
        if b_out
            return 0
        else
            return -1
        end
    else
        return 1
    end
end

function compare_generator(f1, f2, limit)
    a_i = 1
    b_i = 1
    f_a = f1()
    f_b = f2()
    while (a_i < limit) && (b_i < limit)
        res = eval_battle(f_a, f_b)
        if res != -1
            b_i = b_i + 1
            f_b = f2()
        end
        if res != 1
            a_i = a_i + 1
            f_a = f1()
        end
    end
    return (a_i/limit, b_i/limit)
end

function random_team(f::Function, team_size::Int)::Array{Spud}
    team = Array{Spud}(undef, team_size)
    for i in 1:team_size
        team[i] = f()
    end
    return team
end

function df_to_spuds(df::DataFrame)::Array{Spud}
    n = size(df)[1]
    as = Array{Spud}(undef, n)
    for i in 1:n
        as[i] = Spud(df[i, :name], df[i, :h], df[i, :f], df[i, :l], df[i, :r], df[i, :p], df[i, :s])
    end
    return as
end

function cost(a::Spud)::Int64
    return cost(a.h, a.f, a.l, a.p, a.r, a.s)
end

random_team (generic function with 1 method)

In [7]:
function upgrade_spud(sp::Spud)::Spud
    h = sp.h
    f = sp.f
    l = sp.l
    p = sp.p
    r = sp.r
    s = sp.s    
    check_h = (h == MXV) || (cost(h+1,f,l,p,r,s) > MXS)
    check_f = (f == MXV) || (cost(h,f+1,l,p,r,s) > MXS)
    check_l = (l == MXV) || (cost(h,f,l+1,p,r,s) > MXS)
    check_prs = (p+r+s == 3*MXV) || (cost(h,f,l,p+1,r,s) > MXS)
    while !(check_h && check_f && check_l && check_prs)
        rand_i = rand(1:4)
        if rand_i == 1 && !check_h
            h = h+1
        end
        if rand_i == 2 && !check_f
            f = f+1
        end
        if rand_i == 3 && !check_l
            l = l+1
        end
        if rand_i == 4 && !check_prs
            rand_j = rand([1,1,1,1,1,2,2,2,2,3,3,3])
            if rand_j == 1 && p <= MXV
                p += 1
            end
            if rand_j == 2 && r <= MXV
                r += 1
            end
            if rand_j == 3 && s <= MXV
                s += 1
            end
        end            
        check_h = (h == MXV) || (cost(h+1,f,l,p,r,s) > MXS)
        check_f = (f == MXV) || (cost(h,f+1,l,p,r,s) > MXS)
        check_l = (l == MXV) || (cost(h,f,l+1,p,r,s) > MXS)
        check_prs = (p+r+s == 3*MXV) || (cost(h,f,l,p+1,r,s) > MXS)
    end
    return Spud(sp.name, h, f, l, p, r, s)
end



function pick_best_library(bs::Array{Spud})::Spud
    bestscore = -999
    bestf = Spud("",MNV,MNV,MNV,MNV,MNV,MNV)
    df = spuds_to_df(bs)
    hrange = vcat([MNV], df.h, df.h .+ 1, df.f, df.f .+ 1)
    frange = vcat([MNV], df.f, df.f .+ 1, df.h, df.h .+ 1)
    lrange = vcat([MNV], df.l, df.l .+ 1)
    prange = vcat([MNV], df.p, df.p .+ 1)
    rrange = vcat([MNV], df.r, df.r .+ 1)
    srange = vcat([MNV], df.s, df.s .+ 1)
    hrange = sort(unique(hrange))
    frange = sort(unique(frange))
    lrange = sort(unique(lrange))
    prange = sort(unique(prange))
    rrange = sort(unique(rrange))
    srange = sort(unique(srange))
    for h in hrange
        if (cost(h, MNV, MNV, MNV, MNV, MNV) <= MXS)
            for f in frange
                if (cost(h, f, MNV, MNV, MNV, MNV) <= MXS)
                    for l in lrange
                        if (cost(h, f, l, MNV, MNV, MNV) <= MXS)
                            for p in prange
                                if (cost(h, f, l, p, MNV, MNV) <= MXS)
                                    for r in rrange
                                        if (cost(h, f, l, p, r, MNV) <= MXS)
                                            for s in srange
                                                if (cost(h,f,l,p,r,s) <= MXS)
                                                    ff = Spud("",h,f,l,p,r,s)
                                                    score = eval_battle_list(ff, bs)
                                                    if score > bestscore
                                                        bestscore = score
                                                        bestf = ff
                                                    end
                                                end
                                            end
                                        end
                                    end                        
                                end
                            end                        
                        end
                    end
                end
            end
        end
    end
    return upgrade_spud(bestf)
end



pick_best_library (generic function with 1 method)

In [8]:
function pick_ok_counter(bs::Array{Spud})::Spud
    bestscore = -999
    bestf = Spud("",MNV,MNV,MNV,MNV,MNV,MNV)
    df = spuds_to_df(bs)
    hrange = vcat([MNV], df.f .+ 1)
    frange = vcat([MNV], df.h .+ 1)
    lrange = vcat([MNV], df.l .+ 1)
    prange = vcat([MNV], df.p .+ 1)
    rrange = vcat([MNV], df.r .+ 1)
    srange = vcat([MNV], df.s .+ 1)
    hrange = sort(unique(hrange))
    frange = sort(unique(frange))
    lrange = sort(unique(lrange))
    prange = sort(unique(prange))
    rrange = sort(unique(rrange))
    srange = sort(unique(srange))
    for h in hrange
        if (cost(h, MNV, MNV, MNV, MNV, MNV) <= MXS)
            for f in frange
                if (cost(h, f, MNV, MNV, MNV, MNV) <= MXS)
                    for l in lrange
                        if (cost(h, f, l, MNV, MNV, MNV) <= MXS)
                            for p in prange
                                if (cost(h, f, l, p, MNV, MNV) <= MXS)
                                    for r in rrange
                                        if (cost(h, f, l, p, r, MNV) <= MXS)
                                            for s in srange
                                                if (cost(h,f,l,p,r,s) <= MXS)
                                                    ff = Spud("",h,f,l,p,r,s)
                                                    score = eval_battle_list(ff, bs)
                                                    if score > bestscore
                                                        bestscore = score
                                                        bestf = ff
                                                    end
                                                end
                                            end
                                        end
                                    end                        
                                end
                            end                        
                        end
                    end
                end
            end
        end
    end
    return upgrade_spud(bestf)
end


pick_ok_counter (generic function with 1 method)

In [9]:
function pick_best_library2(bs::Array{Spud}, w::Vector{Float64})::Spud
    bestscore = -999.9
    bestf = Spud("",MNV,MNV,MNV,MNV,MNV,MNV)
    df = spuds_to_df(bs)
    hrange = vcat([MNV], df.h, df.h .+ 1, df.f, df.f .+ 1)
    frange = vcat([MNV], df.f, df.f .+ 1, df.h, df.h .+ 1)
    lrange = vcat([MNV], df.l, df.l .+ 1)
    prange = vcat([MNV], df.p, df.p .+ 1)
    rrange = vcat([MNV], df.r, df.r .+ 1)
    srange = vcat([MNV], df.s, df.s .+ 1)
    hrange = sort(unique(hrange))
    frange = sort(unique(frange))
    lrange = sort(unique(lrange))
    prange = sort(unique(prange))
    rrange = sort(unique(rrange))
    srange = sort(unique(srange))
    for h in hrange
        if (cost(h, MNV, MNV, MNV, MNV, MNV) <= MXS)
            for f in frange
                if (cost(h, f, MNV, MNV, MNV, MNV) <= MXS)
                    for l in lrange
                        if (cost(h, f, l, MNV, MNV, MNV) <= MXS)
                            for p in prange
                                if (cost(h, f, l, p, MNV, MNV) <= MXS)
                                    for r in rrange
                                        if (cost(h, f, l, p, r, MNV) <= MXS)
                                            for s in srange
                                                if (cost(h,f,l,p,r,s) <= MXS)
                                                    ff = Spud("",h,f,l,p,r,s)
                                                    score = eval_battle_list2(ff, bs, w) + 0.001 * rand()
                                                    if score > bestscore
                                                        bestscore = score
                                                        bestf = ff
                                                    end
                                                end
                                            end
                                        end
                                    end                        
                                end
                            end                        
                        end
                    end
                end
            end
        end
    end
    return upgrade_spud(bestf)
end

pick_best_library2 (generic function with 1 method)

In [10]:
function subset_library(bs::Array{Spud}, n_hits::Int, w::Vector{Float64}, thres::Float64)::Array{Spud}
    bestf = Array{Spud}(undef, n_hits)
    spud_i = 0
    df = spuds_to_df(bs)
    hrange = vcat([MNV], df.h, df.h .+ 1, df.f, df.f .+ 1)
    frange = vcat([MNV], df.f, df.f .+ 1, df.h, df.h .+ 1)
    lrange = vcat([MNV], df.l, df.l .+ 1)
    prange = vcat([MNV], df.p, df.p .+ 1)
    rrange = vcat([MNV], df.r, df.r .+ 1)
    srange = vcat([MNV], df.s, df.s .+ 1)
    hrange = sort(unique(hrange))
    frange = sort(unique(frange))
    lrange = sort(unique(lrange))
    prange = sort(unique(prange))
    rrange = sort(unique(rrange))
    srange = sort(unique(srange))
    for h in hrange
        if (cost(h, MNV, MNV, MNV, MNV, MNV) <= MXS)
            for f in frange
                if (cost(h, f, MNV, MNV, MNV, MNV) <= MXS)
                    for l in lrange
                        if (cost(h, f, l, MNV, MNV, MNV) <= MXS)
                            for p in prange
                                if (cost(h, f, l, p, MNV, MNV) <= MXS)
                                    for r in rrange
                                        if (cost(h, f, l, p, r, MNV) <= MXS)
                                            for s in srange
                                                if (cost(h,f,l,p,r,s) <= MXS)
                                                    ff = Spud("",h,f,l,p,r,s)
                                                    score = eval_battle_list2(ff, bs, w)
                                                    if score >= thres && spud_i < n_hits
                                                        spud_i += 1
                                                        bestf[spud_i] = upgrade_spud(ff)
                                                    end
                                                end
                                            end
                                        end
                                    end                        
                                end
                            end                        
                        end
                    end
                end
            end
        end
    end
    return unique(bestf[1:spud_i])
end

subset_library (generic function with 1 method)

In [54]:
# form initial library by subsampling indices
library = Array{Spud}(undef, 1000)
spud_i = 0
ss_prob = 0.01

0.01

In [55]:

hrange = MNV:MXV
frange = MNV:MXV
lrange = MNV:MXV
prange = MNV:MXV
rrange = MNV:MXV
srange = MNV:MXV

for h in hrange
    if (cost(h, MXV, MXV, MXV, MXV, MXV) >= MXS) && (cost(h, MNV, MNV, MNV, MNV, MNV) <= MXS)
        for f in frange
            if (cost(h, f, MXV, MXV, MXV, MXV) >= MXS) && (cost(h, f, MNV, MNV, MNV, MNV) <= MXS)
                for l in lrange
                    if (cost(h, f, l, MXV, MXV, MXV) >= MXS) && (cost(h, f, l, MNV, MNV, MNV) <= MXS)
                        for p in prange
                            if (cost(h, f, l, p, MXV, MXV) >= MXS) && (cost(h, f, l, p, MNV, MNV) <= MXS)
                                for r in rrange
                                    if (cost(h, f, l, p, r, MXV) >= MXS) && (cost(h, f, l, p, r, MNV) <= MXS)
                                        for s in srange
                                            if rand() < ss_prob && (cost(h,f,l,p,r,s) <= MXS)
                                                check_f = (h == MXV) || (cost(h+1,f,l,p,r,s) > MXS)
                                                check_h = (f == MXV) || (cost(h,f+1,l,p,r,s) > MXS)
                                                check_l = (l == MXV) || (cost(h,f,l+1,p,r,s) > MXS)
                                                check_prs = (p+r+s == 3*MXV) || (cost(h,f,l,p+1,r,s) > MXS)
                                                if check_h && check_f && check_l && check_prs
                                                    spud_i += 1
                                                    #randname = rand_rename(Spud(" ",h,f,l,p,r,s)).name
                                                    #name = string("#", @sprintf("%i", spud_i), ". ", randname)
                                                    name = ""
                                                    library[spud_i] = Spud(name,h,f,l,p,r,s)
                                                end
                                            end
                                        end
                                    end
                                end                        
                            end
                        end                        
                    end
                end
            end
        end
    end
end


In [56]:
library = unique(library[1:spud_i])
n_spuds = length(library)

714

In [57]:
# for i in 1:n_spuds
#     ff = library[i]
#     randname = rand_rename(ff).name
#     name = string("#", @sprintf("%i", i), ". ", randname)
#     library[i] = Spud(name,ff.h,ff.f,ff.l,ff.p,ff.r,ff.s)
# end
# id_no = Dict(library[i] => i for i in 1:n_spuds)

In [58]:
# Check that there are no ties in spudland
for iter in 1:10000
    i = rand(1:n_spuds)
    j = rand(1:n_spuds)
    if i != j && eval_battle(library[i], library[j]) == 0
        println(library[i])
        println(library[j])
        println()
    end
end

## Compute Nash env

In [59]:
function ffp(nash_env, nits)
    n_nash = length(nash_env)
    i_lose = Array{Int}(undef, (n_nash, n_nash))
    n_lose = Array{Int}(undef, n_nash)
    for i in 1:n_nash
        n_lose[i] = 0
        ff = nash_env[i]
        for j in 1:n_nash
            if eval_battle(ff, nash_env[j]) ==-1
                n_lose[i] += 1
                i_lose[i, n_lose[i]] = j
            end
        end
    end
    counts = [0 for i in 1:n_nash]
    wins = [0 for i in 1:n_nash]
    for i in 1:n_nash
        counts[i] += 1
        for j in 1:n_lose[i]
            i_w = i_lose[i, j]
            wins[i_w]+= 1
        end
    end
    for iter in 1:nits
        wc = 2 .* wins .+ counts
        ind_winners = findall(wc .== maximum(wc))
        i = rand(ind_winners)
        counts[i] += 1
        for j in 1:n_lose[i]
            i_w = i_lose[i, j]
            wins[i_w]+= 1
        end
    end
    return counts
end


ffp (generic function with 1 method)

In [60]:
nash_env = [rand_rename(s) for s in library]
spuds_to_df(nash_env)

Row,name,h,f,l,p,r,s
,String,Int64,Int64,Int64,Int64,Int64,Int64
1,Exquisite System designer Ajiaco,1,2,4,18,9,20
2,Stalwart System designer Brændende kærlighed,1,2,6,20,7,19
3,Dull Technical engineer Okroshka,1,2,10,9,19,16
4,Compassionate System designer Hash browns,1,2,12,11,13,19
5,Zealous Real estate investor Batata vada,1,2,12,13,19,11
6,Limp System designer Aligot,1,2,12,16,13,14
7,Peaceful Real estate investor Batata vada,1,2,12,17,13,13
8,Practical Contractor- general French fries,1,2,14,12,10,20
9,Genuine Soil scientist Kouign patatez,1,2,16,14,11,16


In [61]:
counts = ffp(nash_env, 1000);

In [62]:
nash_env = nash_env[counts .> 10]
spuds_to_df(nash_env)

Row,name,h,f,l,p,r,s
,String,Int64,Int64,Int64,Int64,Int64,Int64
1,Unremarkable Firefighter Truffade,3,5,11,3,4,3
2,Cognizant Paramedic Cacasse à cul nu,3,8,4,2,3,3
3,Elusive Queen mother French fries,4,5,6,1,5,5
4,Understanding Gondolier Clapshot,4,8,7,1,3,1
5,Canny Stunt double Chips and dip,5,5,4,5,2,4
6,Slack Copywriter Chips and dip,5,5,10,1,3,1
7,Ethical Shoeshiner Clapshot,5,6,5,4,1,2
8,Indifferent Funeral director Rappie pie,6,6,2,4,1,3
9,Attractive Impersonator Truffade,7,1,12,4,1,4


In [63]:
counts = ffp(nash_env, 10000)

13-element Vector{Int64}:
 2022
 2006
    1
    1
    1
    1
    1
    1
 1957
 1985
    4
 2028
    5

In [64]:
nash_env = nash_env[counts .> 10]
spuds_to_df(nash_env)

Row,name,h,f,l,p,r,s
,String,Int64,Int64,Int64,Int64,Int64,Int64
1,Unremarkable Firefighter Truffade,3,5,11,3,4,3
2,Cognizant Paramedic Cacasse à cul nu,3,8,4,2,3,3
3,Attractive Impersonator Truffade,7,1,12,4,1,4
4,Uncommunicable Housekeeper Nikujaga,7,7,3,1,2,1
5,Unperceptive Gladiator Hash browns,9,2,7,5,2,2


### Loop this

In [65]:
counts = ffp(nash_env, 100)

5-element Vector{Int64}:
 18
 18
 25
 23
 21

In [66]:
#counts = ffp(nash_env, 10000)

In [67]:
spuds_to_df(nash_env)

Row,name,h,f,l,p,r,s
,String,Int64,Int64,Int64,Int64,Int64,Int64
1,Unremarkable Firefighter Truffade,3,5,11,3,4,3
2,Cognizant Paramedic Cacasse à cul nu,3,8,4,2,3,3
3,Attractive Impersonator Truffade,7,1,12,4,1,4
4,Uncommunicable Housekeeper Nikujaga,7,7,3,1,2,1
5,Unperceptive Gladiator Hash browns,9,2,7,5,2,2


In [68]:
# # don't run this every iteration!!
# nash_env = nash_env[counts .> 10]
# counts = ffp(nash_env, 10000)

In [69]:
ff = rand_rename(pick_best_library2(nash_env, counts./sum(counts)))

Spud("Empowering Syndicalist Chips and dip", 1, 9, 1, 5, 2, 3)

In [70]:
ev = eval_battle_list2(ff, nash_env, counts./sum(counts))

1.0

In [71]:
bestf = subset_library(nash_env, 500, counts./sum(counts), ev-0.05)

4-element Vector{Spud}:
 Spud("", 1, 9, 1, 4, 3, 3)
 Spud("", 1, 9, 1, 5, 2, 3)
 Spud("", 1, 9, 1, 5, 4, 1)
 Spud("", 1, 9, 1, 6, 1, 3)

In [72]:
cc = ffp(bestf, 10000)
println(maximum(cc))
ff2 = rand_rename(bestf[cc .== maximum(cc)][1])

10001


Spud("Self-reliant Puppeteer Papas chorreadas", 1, 9, 1, 6, 1, 3)

In [73]:
eval_battle_list2(ff2, nash_env, counts./sum(counts))

1.0

In [74]:
append!(nash_env, [ff2])
spuds_to_df(nash_env)

Row,name,h,f,l,p,r,s
,String,Int64,Int64,Int64,Int64,Int64,Int64
1,Unremarkable Firefighter Truffade,3,5,11,3,4,3
2,Cognizant Paramedic Cacasse à cul nu,3,8,4,2,3,3
3,Attractive Impersonator Truffade,7,1,12,4,1,4
4,Uncommunicable Housekeeper Nikujaga,7,7,3,1,2,1
5,Unperceptive Gladiator Hash browns,9,2,7,5,2,2
6,Self-reliant Puppeteer Papas chorreadas,1,9,1,6,1,3


### loop

In [75]:
counts = ffp(nash_env, 100000)
println(maximum([eval_battle_list2(ff, nash_env, counts./sum(counts)) for ff in nash_env]))
ff = rand_rename(pick_best_library2(nash_env, counts./sum(counts)))
ev = eval_battle_list2(ff, nash_env, counts./sum(counts))
println(ev)

4.9997000179989205e-5
0.999980001199928


In [76]:
bestf = subset_library(nash_env, 900, counts./sum(counts), ev-0.05)

819-element Vector{Spud}:
 Spud("", 3, 5, 11, 3, 3, 4)
 Spud("", 2, 6, 5, 5, 3, 5)
 Spud("", 2, 6, 5, 3, 4, 6)
 Spud("", 5, 5, 5, 1, 3, 6)
 Spud("", 7, 3, 5, 2, 8, 4)
 Spud("", 9, 3, 5, 2, 4, 6)
 Spud("", 5, 5, 2, 1, 7, 5)
 Spud("", 2, 5, 2, 5, 5, 7)
 Spud("", 7, 4, 4, 4, 4, 4)
 Spud("", 3, 4, 9, 5, 5, 5)
 Spud("", 3, 4, 6, 7, 7, 6)
 Spud("", 5, 4, 7, 2, 4, 6)
 Spud("", 1, 5, 10, 6, 5, 6)
 ⋮
 Spud("", 2, 7, 3, 1, 6, 5)
 Spud("", 6, 5, 4, 2, 3, 4)
 Spud("", 3, 5, 5, 5, 5, 6)
 Spud("", 4, 6, 2, 3, 4, 5)
 Spud("", 4, 6, 2, 2, 5, 4)
 Spud("", 2, 6, 2, 3, 5, 6)
 Spud("", 1, 6, 5, 4, 6, 6)
 Spud("", 3, 3, 11, 8, 5, 6)
 Spud("", 4, 6, 2, 3, 3, 5)
 Spud("", 3, 6, 5, 4, 3, 4)
 Spud("", 3, 5, 5, 5, 6, 5)
 Spud("", 5, 4, 4, 6, 5, 4)

In [77]:
ff

Spud("Steadfast Proctologist Meat and potato pie", 2, 7, 1, 1, 6, 5)

In [78]:
ff = rand_rename(pick_best_library2(nash_env, counts./sum(counts)))
ff

Spud("Idle Percussionist Batata vada", 4, 3, 13, 6, 3, 3)

In [79]:
eval_battle_list2(ff, nash_env, counts./sum(counts))

0.999980001199928

In [80]:
ev

0.999980001199928

In [81]:
ev

0.999980001199928

In [82]:
println(length(bestf))
cc = ffp(bestf, 10000)
println(maximum(cc))

819
618


In [83]:
ff2 = rand_rename(bestf[cc .== maximum(cc)][1])

Spud("Persevering Laundress (also Lavendar) Älplermagronen", 4, 7, 3, 1, 3, 5)

In [84]:
println(eval_battle_list2(ff2, nash_env, counts./sum(counts)))

0.9999400035997841


In [85]:
append!(nash_env, [ff2])

7-element Vector{Spud}:
 Spud("Unremarkable Firefighter Truffade", 3, 5, 11, 3, 4, 3)
 Spud("Cognizant Paramedic Cacasse à cul nu", 3, 8, 4, 2, 3, 3)
 Spud("Attractive Impersonator Truffade", 7, 1, 12, 4, 1, 4)
 Spud("Uncommunicable Housekeeper Nikujaga", 7, 7, 3, 1, 2, 1)
 Spud("Unperceptive Gladiator Hash browns", 9, 2, 7, 5, 2, 2)
 Spud("Self-reliant Puppeteer Papas chorreadas", 1, 9, 1, 6, 1, 3)
 Spud("Persevering Laundress (also Lavendar) Älplermagronen", 4, 7, 3, 1, 3, 5)

In [88]:
for ii in 1:200
    #println()
    print(ii)
    counts = ffp(nash_env, 100000)
    #println(maximum([eval_battle_list2(ff, nash_env, counts./sum(counts)) for ff in nash_env]))
    ff = rand_rename(pick_best_library2(nash_env, counts./sum(counts)))
    ev = eval_battle_list2(ff, nash_env, counts./sum(counts))
    #println(ev)
    bestf = subset_library(nash_env, 900, counts./sum(counts), ev-0.05)
    #println(length(bestf))
    cc = ffp(bestf, 10000)
    #println(maximum(cc))
    ff2 = rand_rename(bestf[cc .== maximum(cc)][1])
    #println(eval_battle_list2(ff2, nash_env, counts./sum(counts)))
    append!(nash_env, [ff2])
end
println()
counts = ffp(nash_env, 100000)
#ff = pick_best_library2(nash_env, counts./sum(counts))
#eval_battle_list2(ff, nash_env, counts./sum(counts))    
nash_env = nash_env[counts .> 10]
counts = ffp(nash_env, 100000);

ff = pick_best_library2(nash_env, counts./sum(counts))
eval_battle_list2(ff, nash_env, counts./sum(counts))

123456789101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899100101102103104105106107108109110111112113114115116117118119120121122123124125126127128129130131132133134135136137138139140141142143144145146147148149150151152153154155156157158159160161162163164165166167168169170171172173174175176177178179180181182183184185186187188189190191192193194195196197198199200


0.057963926015391784

In [89]:
nash_env

183-element Vector{Spud}:
 Spud("Attractive Impersonator Truffade", 7, 1, 12, 4, 1, 4)
 Spud("Self-reliant Puppeteer Papas chorreadas", 1, 9, 1, 6, 1, 3)
 Spud("Jovial Environmental scientist Funeral potatoes", 2, 6, 1, 8, 7, 1)
 Spud("Disinclined Copywriter Chapalele", 8, 3, 6, 3, 2, 4)
 Spud("Inactive Film producer Batata vada", 5, 4, 8, 5, 4, 1)
 Spud("Compassionate Student Maluns", 10, 5, 3, 2, 1, 1)
 Spud("Unsuspecting Juggler Meat and potato pie", 8, 1, 10, 5, 2, 5)
 Spud("Keen Novelist Kouign patatez", 4, 8, 1, 4, 1, 3)
 Spud("Adventurous Ferryman Rumbledethumps", 4, 10, 2, 2, 2, 1)
 Spud("Captivating Housekeeper Suan la tu dou si", 7, 4, 5, 6, 1, 2)
 Spud("Literate Ferryman Chapalele", 7, 9, 1, 1, 1, 1)
 Spud("Unfazed Miller Pitepalt", 1, 9, 1, 5, 4, 1)
 Spud("Exhilarating Drug dealer Potato babka", 11, 1, 7, 7, 1, 4)
 ⋮
 Spud("Laid-back Laundress (also Lavendar) Potato babka", 8, 5, 3, 1, 3, 3)
 Spud("Empathetic Dictator Truffade", 2, 5, 15, 5, 6, 1)
 Spud("Passionless Party-l

## beware

In [45]:
CSV.write("spuds_nash_stage01.csv", spuds_to_df(nash_env))

"spuds_nash_stage01.csv"

## adjust subset threshold

In [110]:
counts = ffp(nash_env, 100000)
println(maximum([eval_battle_list2(ff, nash_env, counts./sum(counts)) for ff in nash_env]))

0.014511676118525482


In [111]:

for ii in 1:100
    #println()
    println(ii)
    counts = ffp(nash_env, 100000)
    #println(maximum([eval_battle_list2(ff, nash_env, counts./sum(counts)) for ff in nash_env]))
    ff = rand_rename(pick_best_library2(nash_env, counts./sum(counts)))
    ev = eval_battle_list2(ff, nash_env, counts./sum(counts))
    #println(ev)
    bestf = subset_library(nash_env, 900, counts./sum(counts), ev-0.01)
    println(length(bestf))
    cc = ffp(bestf, 10000)
    #println(maximum(cc))
    ff2 = rand_rename(bestf[cc .== maximum(cc)][1])
    println(ff2)
    #println(eval_battle_list2(ff2, nash_env, counts./sum(counts)))
    append!(nash_env, [ff2])
end
println()
counts = ffp(nash_env, 100000)
#ff = pick_best_library2(nash_env, counts./sum(counts))
#eval_battle_list2(ff, nash_env, counts./sum(counts))    
nash_env = nash_env[counts .> 10]
counts = ffp(nash_env, 100000);

ff = pick_best_library2(nash_env, counts./sum(counts))
eval_battle_list2(ff, nash_env, counts./sum(counts))

1
14
Spud("Independent Dictator Boxty", 4, 4, 10, 6, 4, 1)
2
22
Spud("Adroit Neurologist Clapshot", 3, 8, 1, 2, 4, 3)
3
51
Spud("Self-sufficient Ferryman Papas chorreadas", 7, 6, 4, 1, 2, 2)
4
5
Spud("Appreciative Quilter Clapshot", 4, 8, 1, 5, 1, 2)
5
39
Spud("Self-assured Coast guard Boxty", 1, 7, 2, 6, 1, 6)
6
2
Spud("Uninspiring Lumberjack Ajiaco", 11, 3, 4, 3, 1, 3)
7
32
Spud("Well-informed Puppeteer Älplermagronen", 2, 8, 2, 5, 4, 1)
8
11
Spud("Torpid Juggler Suan la tu dou si", 7, 1, 12, 3, 5, 1)
9
2
Spud("Lackluster Lumberjack Kouign patatez", 10, 1, 8, 3, 2, 5)
10
6
Spud("Thoughtful Chimney sweep Rumbledethumps", 7, 5, 5, 3, 1, 2)
11
8
Spud("Impotent Impersonator French fries", 6, 1, 14, 4, 4, 2)
12
6
Spud("Inspiring Party-leader Boulangère potatoes", 2, 5, 15, 8, 3, 1)
13
15
Spud("Unconstrained Juggler Pommes Anna", 6, 3, 9, 6, 1, 2)
14
23
Spud("Introspective Dictator French fries", 9, 1, 9, 6, 3, 1)
15
6
Spud("Benevolent Copywriter Maluns", 7, 4, 5, 2, 4, 3)
16
2
Spud("Color

0.033284834472724276

In [112]:
nash_env

346-element Vector{Spud}:
 Spud("Jovial Environmental scientist Funeral potatoes", 2, 6, 1, 8, 7, 1)
 Spud("Disinclined Copywriter Chapalele", 8, 3, 6, 3, 2, 4)
 Spud("Compassionate Student Maluns", 10, 5, 3, 2, 1, 1)
 Spud("Keen Novelist Kouign patatez", 4, 8, 1, 4, 1, 3)
 Spud("Literate Ferryman Chapalele", 7, 9, 1, 1, 1, 1)
 Spud("Unfazed Miller Pitepalt", 1, 9, 1, 5, 4, 1)
 Spud("Exhilarating Drug dealer Potato babka", 11, 1, 7, 7, 1, 4)
 Spud("Unsympathetic Queen dowager Suan la tu dou si", 11, 4, 4, 1, 1, 1)
 Spud("Unresponsive Nanny (also nursemaid) Rappie pie", 8, 6, 4, 1, 1, 1)
 Spud("Unappealing Drug dealer Okroshka", 12, 2, 5, 2, 4, 2)
 Spud("Generous Ferryman Hash browns", 8, 8, 1, 1, 1, 1)
 Spud("Mindful Student Ajiaco", 10, 6, 1, 3, 1, 1)
 Spud("Stimulating Ferryman Hash browns", 5, 9, 2, 2, 2, 1)
 ⋮
 Spud("Lethargic Investment broker Rewena bread", 7, 3, 7, 1, 5, 4)
 Spud("Erudite Housekeeper Pommes Anna", 8, 5, 3, 2, 3, 2)
 Spud("Charmed Cardinal Brændende kærlighed", 5

In [113]:
nash_copy = unique([Spud("", s.h, s.f, s.l, s.p, s.r, s.s) for s in nash_env])

346-element Vector{Spud}:
 Spud("", 2, 6, 1, 8, 7, 1)
 Spud("", 8, 3, 6, 3, 2, 4)
 Spud("", 10, 5, 3, 2, 1, 1)
 Spud("", 4, 8, 1, 4, 1, 3)
 Spud("", 7, 9, 1, 1, 1, 1)
 Spud("", 1, 9, 1, 5, 4, 1)
 Spud("", 11, 1, 7, 7, 1, 4)
 Spud("", 11, 4, 4, 1, 1, 1)
 Spud("", 8, 6, 4, 1, 1, 1)
 Spud("", 12, 2, 5, 2, 4, 2)
 Spud("", 8, 8, 1, 1, 1, 1)
 Spud("", 10, 6, 1, 3, 1, 1)
 Spud("", 5, 9, 2, 2, 2, 1)
 ⋮
 Spud("", 7, 3, 7, 1, 5, 4)
 Spud("", 8, 5, 3, 2, 3, 2)
 Spud("", 5, 2, 12, 7, 7, 1)
 Spud("", 9, 1, 9, 4, 3, 3)
 Spud("", 7, 1, 11, 1, 8, 7)
 Spud("", 3, 8, 1, 6, 2, 1)
 Spud("", 6, 3, 8, 3, 5, 3)
 Spud("", 8, 4, 4, 6, 1, 2)
 Spud("", 7, 1, 11, 8, 1, 7)
 Spud("", 7, 1, 12, 6, 2, 1)
 Spud("", 1, 9, 1, 2, 3, 5)
 Spud("", 3, 5, 15, 2, 4, 2)

In [114]:
CSV.write("spuds_nash_stage04.csv", spuds_to_df(nash_env))

"spuds_nash_stage04.csv"

### refine weights

In [ ]:
eps = 0.01/length(library)
nits = 40000
sub_iter = 2000
w = counts./sum(counts)

In [ ]:
n_nash = length(nash_env)
i_lose = Array{Int}(undef, (n_nash, n_nash))
n_lose = Array{Int}(undef, n_nash)
for i in 1:n_nash
    n_lose[i] = 0
    ff = nash_env[i]
    for j in 1:n_nash
        if eval_battle(ff, nash_env[j]) ==-1
            n_lose[i] += 1
            i_lose[i, n_lose[i]] = j
        end
    end
end

In [ ]:
scores = [0.0 for i in 1:n_nash]
for i in 1:n_nash
    for j in 1:n_lose[i]
        i_w = i_lose[i, j]
        scores[i_w]+= w[i]
    end
end

In [ ]:
previous_score = maximum(scores)
for iter in 1:nits
    ind_winners = findall(scores .== maximum(scores))
    i = rand(ind_winners)
    w[i] += eps
    for j in 1:n_lose[i]
        i_w = i_lose[i, j]
        scores[i_w]+= eps
    end
    if mod1(iter, sub_iter) == 1
        scores = scores./sum(w)
        w = w./sum(w)
        if maximum(scores) > previous_score
            eps = eps/2
            println(eps)
            println()
            println()
        end
        previous_score = maximum(scores)
        mxv, mxi = findmax(scores)
        print(nash_env[mxi])
        print(" w:")
        print(w[mxi])
        print(" s:")
        print(scores[mxi])
        println()
    end
end


In [ ]:
maximum([eval_battle_list2(ff, nash_env, w) for ff in nash_env])

In [ ]:
ff = pick_best_library2(nash_env, w)
println(ff)
eval_battle_list2(ff, nash_env, w)

## Old code

In [ ]:
function nashprox2(size, start_size, ns)
    nash_env = Array{Spud}(undef, size)
    nash_env[1:start_size] = random_team(() -> rand(library), start_size)
    for j in (start_size+1):size
        if j < ns
            ff = pick_ok_counter(nash_env[1:(j-1)])
            nash_env[j] = ff
        else
            ff = pick_ok_counter(randsubseq(nash_env[1:(j-1)], ns/j))
            nash_env[j] = ff                
        end
    end
    return nash_env
end

function nashprox(size, start_size, ns)
    nash_env = Array{Spud}(undef, size)
    nash_env[1:start_size] = random_team(() -> rand(library), start_size)
    for j in (start_size+1):size
        if j < ns
            ff = pick_best_library(nash_env[1:(j-1)])
            nash_env[j] = ff
        else
            ff = pick_best_library(randsubseq(nash_env[1:(j-1)], ns/j))
            nash_env[j] = ff                
        end
    end
    return nash_env
end

In [ ]:
@time nash_env = nashprox(4000, 10, 100)
nash_env = unique(nash_env)
#inds_nash = [id_no[ff] for ff in nash_env];
#length(inds_nash)
n_nash = length(nash_env)

In [ ]:
# Verify 3-explotability
for i in 1:1000
    #inds = sort([rand(inds_nash) for i in 1:3])
    #rt = library[inds]
    rt = random_team(() -> rand(nash_env), 3)
    ff = pick_best_library(rt)
    ev = eval_battle_list(ff, rt)
    if ev < 3
        println(rt)
    end
end


In [ ]:
# Verify 4-explotability
for i in 1:1000
    #inds = sort([rand(inds_nash) for i in 1:3])
    #rt = library[inds]
    rt = random_team(() -> rand(nash_env), 4)
    ff = pick_best_library(rt)
    ev = eval_battle_list(ff, rt)
    if ev < 4
        println(rt)
        println(ev)
    end
end


In [ ]:
#inds = sort([rand(1:n_spuds) for i in 1:5])
#rt = library[inds]

In [ ]:
# rt = library[[1403,1902,2283,2397]]
# rt

In [ ]:
# ff = pick_best(library, rt)
# ind = [i for i in 1:n_spuds if library[i] == ff][1]
# print(ff)
# print(" ")
# eval_battle_list(ff, rt)

## Fictitious Play to compute Nash distribution

In [ ]:
@time nash_env = nashprox(4000, 10, 100)
nash_env = unique(nash_env)
#inds_nash = [id_no[ff] for ff in nash_env];
#length(inds_nash)
n_nash = length(nash_env)

In [ ]:
i_lose = Array{Int}(undef, (n_nash, n_nash))
n_lose = Array{Int}(undef, n_nash)
for i in 1:n_nash
    n_lose[i] = 0
    ff = nash_env[i]
    for j in 1:n_nash
        if eval_battle(ff, nash_env[j]) ==-1
            n_lose[i] += 1
            i_lose[i, n_lose[i]] = j
        end
    end
end

In [ ]:
# initialize FP with 1 of each
counts = [0 for i in 1:n_nash]
wins = [0 for i in 1:n_nash]
for i in 1:n_nash
    counts[i] += 1
    for j in 1:n_lose[i]
        i_w = i_lose[i, j]
        wins[i_w]+= 1
    end
end

In [ ]:
nits = 500000
for iter in 1:nits
    ind_winners = findall(wins .== maximum(wins))
    for i in ind_winners
        counts[i] += 1
        for j in 1:n_lose[i]
            i_w = i_lose[i, j]
            wins[i_w]+= 1
        end
    end
    if mod1(iter, 10000) == 1
        mxv, mxi = findmax(wins)
        print(nash_env[mxi])
        print(" n:")
        print(counts[mxi])
        print(" w:")
        print(wins[mxi])
        println()
    end
end

w = counts./sum(counts)
println(maximum([eval_battle_list2(ff, nash_env, w) for ff in nash_env]))
sum(counts)

In [ ]:
ff = pick_best_library2(nash_env, w)
println(ff)
eval_battle_list2(ff, nash_env, w)

In [ ]:
inds_sel = sortperm(-w)[1:500]
tab = spuds_to_df(nash_env[inds_sel])
tab[!, :w] = w[inds_sel]
tab

In [ ]:
# Verify explotability
for i in 1:10000
    inds = sort([rand(inds_sel[1:120]) for i in 1:3])
    rt = nash_env[inds]
    ff = pick_best(library, rt)
    ev = eval_battle_list(ff, rt)
    if ev < 3
        println(spuds_to_df(rt))
        println(w[inds])
        println(ev)
    end
end

In [ ]:
rt = nash_env[sort([rand(1:n_nash) for i in 1:4])]

In [ ]:
ff = pick_best_rdmly(nash_env[inds_sel], rt, 99)
ev = eval_battle_list(ff, rt)
println(ff)
println(ev)
eval_battle_list2(ff, nash_env, w)

In [ ]:
ff = pick_best_rdmly(library, rt, 99)
ev = eval_battle_list(ff, rt)
println(ff)
println(ev)
eval_battle_list2(ff, nash_env, w)

In [ ]:
ff = rand(library)
println(ff)
eval_battle_list2(ff, nash_env, w)

In [ ]:
#cost(4,4,9,1,4,2)

In [ ]:
#rt = library[[15918,21020,26191]]
#eval_battle(Spud("",4,4,9,1,4,1), rt[1])

In [ ]:
#sum(sort(-w)[1:450])

In [ ]:
#sum(w)

In [ ]:
# # FP is slow to converge in the end
# eps = 0.01/length(library)
# nits = 20000
# scores = wins./sum(counts)
# w = counts./sum(counts)
# 0

In [ ]:
# previous_score = maximum(scores)
# for iter in 1:nits
#     ind_winners = findall(wins .== maximum(wins))
#     i = rand(ind_winners)
#     w[i] += eps
#     for j in 1:n_lose[i]
#         i_w = i_lose[i, j]
#         scores[i_w]+= eps
#     end
#     if mod1(iter, 2000) == 1
#         scores = scores./sum(w)
#         w = w./sum(w)
#         if maximum(scores) > previous_score
#             eps = eps/2
#             println(eps)
#             println()
#             println()
#         end
#         previous_score = maximum(scores)
#         mxv, mxi = findmax(wins)
#         print(library[mxi])
#         print(" n:")
#         print(w[mxi])
#         print(" w:")
#         print(scores[mxi])
#         println()
#         mxv, mxi = findmax(counts)
#         print(library[mxi])
#         print(" n:")
#         print(w[mxi])
#         print(" w:")
#         print(scores[mxi])
#         println()
#         println(maximum([eval_battle_list2(ff, library, w) for ff in library]))
#         println()
#     end
# end
